# Human keyboard
Remember to click on the bottom bar for the keyboard input to work.\
<img src="click.png" alt="drawing" width="500"/>

In [4]:
import gym
import keyboard
env = gym.make("MountainCar-v0")
env._max_episode_steps = 1000

# Play
state = env.reset()  # [position, velocity]
done = False
score = 0

while not done:
    # Read keyboard input
    int_action = 1  # No push

    if keyboard.is_pressed('right'):
        int_action = 2  # right push
    elif keyboard.is_pressed('left'):
        int_action = 0  # left push

    # Send input to game
    state, reward, done, info = env.step(int_action)

    score += reward

    # Render new frame
    env.render()
print(score)
env.close()

-298.0


# Train AI

In [ ]:
import gym
import neat
import os
import math
import pickle

def convert_to_int_action(action):
    int_action = 1  # No push

    if action[0] > 0.5:
        int_action = 2  # Right push
    elif action[1] > 0.5:
        int_action = 0  # Left push
    return int_action

def ai_train(genomes, config):
    nets = []  # Neural networks/brain
    ge = []  # Genes

    # Setting up AI
    for _, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)  # Getting values from txt
        nets.append(net)  # Appending every Neural Network to array
        g.fitness = 0
        ge.append(g)  # Appending every gene to array.

    # training every agent
    for g, net in zip(ge, nets):
        # Start game
        state = env.reset()
        done = False

        # Variables for calculating fitness
        high_vel = -math.inf
        low_vel = math.inf
        elapsed_steps = 0

        while not done:
            action = net.activate((state))  # send state to AI

            int_action = convert_to_int_action(action)
            state, reward, done, info = env.step(int_action)
            g.fitness += reward
            pos, vel = state
            elapsed_steps += 1

            if vel > high_vel:
                high_vel = vel
            if vel < low_vel:
                low_vel = vel

# Environment setup
env = gym.make("MountainCar-v0")
env._max_episode_steps = 1000

# Getting file with values
config_path = os.path.abspath("config-feedforward.txt")
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)
p = neat.Population(config)

p.add_reporter(neat.StdOutReporter(True))  # printing stats

num_training_generations = 10
winner = p.run(ai_train, num_training_generations)
print(winner)

# Save best gene in Pickle file
file = open('winner_neat', 'wb')
pickle.dump(winner, file)
file.close()

# Close window
if you force interupt the game, the window will still be open. Run this to close the window

In [3]:
env.close()

: 